In [ ]:
# !pip -qqq install pip --progress-bar off
# !pip -qqq install langchain==0.1.17 --progress-bar off
# !pip -qqq install llama-parse==0.1.3 --progress-bar off
# !pip -qqq install qdrant-client==1.9.1  --progress-bar off
# !pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
# !pip -qqq install fastembed==0.2.7 --progress-bar off
# !pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-aiplatform 1.48.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but y

In [10]:
import os
import textwrap
from pathlib import Path

# from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama



def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [2]:
!mkdir data
!gdown 1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9 -O "data/meta-earnings.pdf"

mkdir: cannot create directory ‘data’: File exists
Traceback (most recent call last):
  File "/home/harshb/miniconda3/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/home/harshb/miniconda3/lib/python3.10/site-packages/gdown/cli.py", line 156, in main
    filename = download(
  File "/home/harshb/miniconda3/lib/python3.10/site-packages/gdown/download.py", line 259, in download
    filename_from_url = m.groups()[0]
AttributeError: 'NoneType' object has no attribute 'groups'


## Document Parsing

In [ ]:
!pip install PyMuPDF markdownify

In [1]:
import fitz  # PyMuPDF
from markdownify import markdownify as md

In [11]:
def pdf_to_markdown(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Initialize an empty string for storing the content
    pdf_text = ""

    # Iterate through each page
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        pdf_text += page.get_text("text")

    # Convert the text to markdown
    markdown_text = md(pdf_text)

    return markdown_text

In [12]:
# parsed_doc = llama_parse_documents[0]
parsed_doc = pdf_to_markdown("data/meta-earnings.pdf")

In [14]:
Markdown(parsed_doc[:4096])

Meta Reports First Quarter 2024 Results
MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter 
ended March 31, 2024\.
"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is 
another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making 
steady progress building the metaverse as well."
First Quarter 2024 Financial Highlights
Three Months Ended March 31,
% Change
In millions, except percentages and per share amounts
2024
2023
Revenue
$ 
36,455 
$ 
28,645 
 
27 %
Costs and expenses
 
22,637 
 
21,418 
 
6 %
Income from operations
$ 
13,818 
$ 
7,227 
 
91 %
Operating margin
 
38 %
 
25 %
Provision for income taxes
$ 
1,814 
$ 
1,598 
 
14 %
Effective tax rate
 
13 %
 
22 %
Net income
$ 
12,369 
$ 
5,709 
 
117 %
Diluted earnings per share (EPS)
$ 
4\.71 
$ 
2\.20 
 
114 %
First Quarter 2024 Operational and Other Financial Highlights
•
Family daily active people (DAP) – DAP was 3\.24 billion on average for March 2024, an increase of 7% year\-over\-
year.
•
Ad impressions – Ad impressions delivered across our Family of Apps increased by 20% year\-over\-year. 
•
Average price per ad – Average price per ad increased by 6% year\-over\-year.
•
Revenue – Total revenue and revenue on a constant currency basis were $36\.46 billion and $36\.35 billion, respectively, 
both of which increased by 27% year\-over\-year.
•
Costs and expenses – Total costs and expenses were $22\.64 billion, an increase of 6% year\-over\-year. 
•
Capital expenditures – Capital expenditures, including principal payments on finance leases, were $6\.72 billion. 
•
Capital return program – Share repurchases were $14\.64 billion of our Class A common stock and dividends 
payments were $1\.27 billion.
•
Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were 
$58\.12 billion as of March 31, 2024\. Free cash flow was $12\.53 billion.
•
Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year\-over\-year. 
1
CFO Outlook Commentary
We expect second quarter 2024 total revenue to be in the range of $36\.5\-39 billion. Our guidance assumes foreign currency is a 
1% headwind to year\-over\-year total revenue growth, based on current exchange rates.
We expect full\-year 2024 total expenses to be in the range of $96\-99 billion, updated from our prior outlook of $94\-99 billion 
due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully 
year\-over\-year due to our ongoing product development efforts and our investments to further scale our ecosystem. 
We anticipate our full\-year 2024 capital expenditures will be in the range of $35\-40 billion, increased from our prior range of 
$30\-37 billion as we continue to accelerate our infrastructure investments to support our artificial intelligence (AI) roadmap. 
While we are not providing guidance for years beyond 2024, we expect capital expenditures will continue to increase next year 
as we invest aggressively to support our ambitious AI research and product development efforts.
Absent any changes to our tax landscape, we expect our full\-year 2024 tax rate to be in the mid\-teens.
In addition, we continue to monitor an active regulatory landscape, including the increasing legal and regulatory headwinds in 
the EU and the U.S. that could significantly impact our business and our financial results.
Q1 was a good start to the year. We're seeing strong momentum within our Family of Apps and are making important progress 
on our longer\-term AI and Reality Labs initiatives that have the potential to transform the way people interact with our services 
over the coming years.
2
Webcast and Conference Call Information
Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings 
conference call can be accessed at investor.fb.com, 

In [16]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc)

## Vector Embeddings

In [17]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

26

In [19]:
print(docs[0].page_content)

Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

First Quarter 2024 Financial Highlights

Three Months Ended March 31, 2024 
 Three Months Ended March 31, 2023 
 % Change 
 
 Revenue 
 $36,455 million 
 $28,645 million 
 27% 
 
 Costs and expenses 
 $22,637 million 
 $21,418 million 
 6% 
 
 Income from operations 
 $13,818 million 
 $7,227 million 
 91% 
 
 Operating margin 
 38% 
 25% 
 
 
 Provision for income taxes 
 $1,814 million 
 $1,598 million 
 14% 
 
 Effective tax rate 
 13% 
 22% 
 
 
 Net income 
 $12,369 million

In [10]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [12]:
%%time
query = "What is the most important innovation from Meta?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 966 ms, sys: 7.28 ms, total: 973 ms
Wall time: 567 ms


In [13]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

score: 0.6271389050410733
--------------------------------------------------------------------------------

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year,

score: 0.6101042835235578
--------------------------------------------------------------------------------

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the

In [14]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 933 ms, sys: 0 ns, total: 933 ms
Wall time: 568 ms


In [15]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

id: f8d12cb72ba84bb6ab52fdde4514e345

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

--------------------------------------------------------------------------------

id: bebbbeffc58f4987bd8aa3f89ee9bece

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year,

--------------------------------------------------------------------------------

id: 2dfa988df69949d8a2196d8ce991f952

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the

## Reranking

In [1]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

# Ensure flashrank is installed
try:
    from flashrank import Ranker, RerankRequest
except ImportError:
    raise ImportError(
        "Could not import flashrank python package. "
        "Please install it with `pip install flashrank`."
    )

# Call update_forward_refs to resolve forward references
FlashrankRerank.update_forward_refs()

compressor = FlashrankRerank()

In [16]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [17]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 31.7 s, sys: 324 ms, total: 32 s
Wall time: 5.55 s


3

In [18]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: bebbbeffc58f4987bd8aa3f89ee9bece

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year,

score: [0.034220486879348755, 0.9561359882354736]
--------------------------------------------------------------------------------

id: 2dfa988df69949d8a2196d8ce991f952

text: Meta First Quarter 2024 Results

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year,

score: [0.034220486879348755, 0.9561359882354736]
--------------------------------------------------------------------------------

id: 32e07a8b023d4e569a9c728141fbe8f7

text: We anticipate our full-year 2024 capital expenditures will be in the range of $35-40 bil

## Q&A Over Document

In [20]:
llm = Ollama(temperature=0, model="llama3")

In [21]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [23]:
%%time
response = qa.invoke("What is the most significant innovation from Meta?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: We anticipate our full-year 2024 capital expenditures will be in the range of $35-40 billion, increased from our prior range of $30-37 billion as we continue to accelerate our infrastructure investments to support our artificial intelligence (AI) roadmap. While we are not providing guidance for years beyond 2024, we expect capital expenditures will continue to increase next year as we invest aggressively to support our ambitious AI research and product development efforts.

Absent any changes to our tax landscape, we expect our full-year 2024 tax rate to be in the mid-teens.

In addition, we continue to monitor an active regulatory landscape, including the increasing legal and regulat

In [24]:
print_response(response)

The most significant innovation from Meta mentioned in the text is the new version of Meta AI with
Llama 3, which is a step towards building the world's leading AI.

Additional helpful information:

* The company is investing aggressively in its artificial intelligence (AI) roadmap and expects
capital expenditures to continue increasing next year.
* The company is also making progress on its longer-term AI and Reality Labs initiatives that have
the potential to transform the way people interact with their services over the coming years.


In [25]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)

In [26]:
%%time
response = qa.invoke("What is the revenue for 2024 and % change?")

Running pairwise ranking..
CPU times: user 33 s, sys: 44.5 ms, total: 33.1 s
Wall time: 2min


In [27]:
Markdown(response["result"])

According to the provided information, the GAAP revenue for Q1 2024 is $36,455.

The year-over-year change in GAAP revenue is 27%.

Here's a breakdown of the revenue:

* GAAP revenue: $36,455 (Q1 2024)
* Foreign exchange effect on 2024 revenue using 2023 rates: ($106) (offsets the actual revenue)
* Revenue excluding foreign exchange effect: $36,349
* GAAP advertising revenue: $35,635 (Q1 2024)
* Advertising revenue excluding foreign exchange effect: $35,530

Note that these figures are based on the provided information and may not reflect the full-year 2024 revenue guidance of $36.5-39 billion mentioned in the CFO Outlook Commentary.

In [ ]:
%%time
response = qa.invoke("What is the revenue for 2023?")

Running pairwise ranking..
CPU times: user 2.53 s, sys: 6.54 ms, total: 2.53 s
Wall time: 7.32 s


In [ ]:
print_response(response)

The revenue for 2023 is $28,645.

Additional information: This is a 27% increase from the previous year, and the revenue excluding
foreign exchange effect is also $28,645.


In [ ]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2024? Calculate the answer"
)

Running pairwise ranking..
CPU times: user 2.68 s, sys: 9.44 ms, total: 2.69 s
Wall time: 7.01 s


In [ ]:
print_response(response)

Based on the provided information, the revenue for Q1 2024 is $36,455 million, and the costs and
expenses are $22,637 million. To calculate the revenue minus the costs and expenses, we can subtract
the costs and expenses from the revenue:

$36,455 million (revenue) - $22,637 million (costs and expenses) = $13,818 million

So, the revenue minus the costs and expenses for Q1 2024 is $13,818 million.

Additional helpful information:

* This amount, $13,818 million, is also referred to as the income from operations.
* The operating margin is 38%, which is calculated by dividing the income from operations by the
revenue and multiplying by 100.


In [ ]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2023? Calculate the answer"
)

Running pairwise ranking..
CPU times: user 2.29 s, sys: 9.6 ms, total: 2.3 s
Wall time: 9 s


In [ ]:
print_response(response)

Based on the provided information, we can calculate the revenue minus costs and expenses for 2023 as
follows:

Revenue (2023): $28,645
Income from operations (2023): $7,227

To find the costs and expenses, we can subtract the income from operations from the revenue:

Costs and expenses (2023) = Revenue - Income from operations
= $28,645 - $7,227
= $21,418

So, the revenue minus costs and expenses for 2023 is $7,227.

Additional helpful information:

* The company's revenue has increased by 27% year-over-year, from $28,645 in 2023 to $36,455 in
2024.
* The company's income from operations has also increased, from $7,227 in 2023 to $13,818 in 2024.


In [ ]:
%%time
response = qa.invoke("What is the expected revenue for the second quarter of 2024?")

Running pairwise ranking..
CPU times: user 2.59 s, sys: 9.42 ms, total: 2.6 s
Wall time: 6.58 s


In [ ]:
Markdown(response["result"])

**Answer:** The expected revenue for the second quarter of 2024 is in the range of $36.5-39 billion.

**Additional information:** This guidance assumes a 1% headwind to year-over-year total revenue growth due to foreign currency exchange rates.

In [ ]:
%%time
response = qa.invoke("What is the overall outlook of Q1 2024?")

Running pairwise ranking..
CPU times: user 3.44 s, sys: 12.9 ms, total: 3.45 s
Wall time: 8.28 s


In [ ]:
print_response(response)

**Overall Outlook of Q1 2024:**

The overall outlook of Q1 2024 is positive. According to Mark Zuckerberg, "It's been a good start to
the year." The company has reported strong financial results, with revenue increasing by 27% year-
over-year to $36.46 billion. Net income has also increased by 117% year-over-year to $12.37 billion.

**Additional Highlights:**

* Family daily active people (DAP) increased by 7% year-over-year to 3.24 billion.
* Ad impressions increased by 20% year-over-year.
* Average price per ad increased by 6% year-over-year.
* Capital expenditures were $6.72 billion, and free cash flow was $12.53 billion.
* The company has also reported a strong capital return program, with share repurchases of $14.64
billion and dividend payments of $1.27 billion.


## References

- [Meta Reports First Quarter 2024 Results](https://s21.q4cdn.com/399680738/files/doc_financials/2024/q1/Meta-03-31-2024-Exhibit-99-1_FINAL.pdf)